# Random Forest Regression

## Importing the libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import math
import gc

## Importing the dataset

In [2]:
train = pd.read_csv('worked/train_worked.csv')

In [3]:
train.head(2)

,ID,location,target,max_temp,min_temp,mean_temp,std_temp,var_temp,median_temp,ptp_temp,...,var_wind_spd,median_wind_spd,ptp_wind_spd,max_atmos_press,min_atmos_press,mean_atmos_press,std_atmos_press,var_atmos_press,median_atmos_press,ptp_atmos_press
0,ID_train_0,C,45.126304,27.208333,19.275000,22.299527,2.594011,6.728895,21.358333,7.933333,...,0.155268,0.666250,1.355000,87.871667,87.614167,87.752434,0.070374,0.004952,87.762083,0.257500
1,ID_train_1,D,79.131702,33.616667,17.983333,24.679063,4.266955,18.206903,23.791667,15.633333,...,0.290736,0.744167,2.760833,90.725000,90.056667,90.429924,0.156000,0.024336,90.429167,0.668333


In [11]:
train = pd.read_csv('worked/train_worked.csv')
y_train = train['target'].values
y_hold = train['ID'].values
train.pop('ID')
train.pop('target')
catFeatures = ['location']
train = pd.get_dummies(train, columns=catFeatures)
train['ID'] = y_hold
train['target'] = y_train
x_train = train.iloc[:, :-2].values
y_train = train.iloc[:, -2:]





In [18]:
test = pd.read_csv('worked/test_worked.csv',  index_col= 'ID')
test.pop('target')
catFeatures = ['location']
test = pd.get_dummies(test, columns=catFeatures)
x_test = test.iloc[:, :].values

In [19]:
test.head(5)

,max_temp,min_temp,mean_temp,std_temp,var_temp,median_temp,ptp_temp,max_precip,min_precip,mean_precip,...,mean_atmos_press,std_atmos_press,var_atmos_press,median_atmos_press,ptp_atmos_press,location_A,location_B,location_C,location_D,location_E
ID,,,,,,,,,,,,,,,,,,,,,
ID_test_0,26.858333,17.050000,21.389532,2.731833,7.462909,20.908333,9.808333,5.728,0.0,0.204165,...,87.748506,0.132764,0.017626,87.752500,0.546667,0,0,1,0,0
ID_test_1,26.991667,17.275000,21.302824,2.289268,5.240748,20.716667,9.716667,15.831,0.0,0.208719,...,87.936963,0.113756,0.012940,87.928333,0.609167,0,0,1,0,0
ID_test_10,29.850000,17.108333,22.144421,2.943047,8.661528,21.458333,12.741667,1.327,0.0,0.040190,...,90.521357,0.136316,0.018582,90.536667,0.670000,0,0,0,1,0
ID_test_100,27.325000,18.633333,22.361639,2.480003,6.150415,21.616667,8.691667,3.604,0.0,0.035686,...,87.754848,0.125583,0.015771,87.726667,0.518333,0,0,1,0,0
ID_test_1000,31.091667,16.908333,23.560243,4.136978,17.114585,22.825000,14.183333,0.000,0.0,0.000000,...,90.475030,0.161193,0.025983,90.487273,0.692500,0,0,0,1,0


In [55]:
train.columns

Index(['max_temp', 'min_temp', 'mean_temp', 'std_temp', 'var_temp',
       'median_temp', 'ptp_temp', 'max_precip', 'min_precip', 'mean_precip',
       'std_precip', 'var_precip', 'median_precip', 'ptp_precip',
       'max_rel_humidity', 'min_rel_humidity', 'mean_rel_humidity',
       'std_rel_humidity', 'var_rel_humidity', 'median_rel_humidity',
       'ptp_rel_humidity', 'max_wind_dir', 'min_wind_dir', 'mean_wind_dir',
       'std_wind_dir', 'var_wind_dir', 'median_wind_dir', 'ptp_wind_dir',
       'max_wind_spd', 'min_wind_spd', 'mean_wind_spd', 'std_wind_spd',
       'var_wind_spd', 'median_wind_spd', 'ptp_wind_spd', 'max_atmos_press',
       'min_atmos_press', 'mean_atmos_press', 'std_atmos_press',
       'var_atmos_press', 'median_atmos_press', 'ptp_atmos_press',
       'location_A', 'location_B', 'location_C', 'location_D', 'location_E',
       'ID', 'target'],
      dtype='object')

In [20]:
test.shape

(5035, 47)

In [21]:
test.head()

,max_temp,min_temp,mean_temp,std_temp,var_temp,median_temp,ptp_temp,max_precip,min_precip,mean_precip,...,mean_atmos_press,std_atmos_press,var_atmos_press,median_atmos_press,ptp_atmos_press,location_A,location_B,location_C,location_D,location_E
ID,,,,,,,,,,,,,,,,,,,,,
ID_test_0,26.858333,17.050000,21.389532,2.731833,7.462909,20.908333,9.808333,5.728,0.0,0.204165,...,87.748506,0.132764,0.017626,87.752500,0.546667,0,0,1,0,0
ID_test_1,26.991667,17.275000,21.302824,2.289268,5.240748,20.716667,9.716667,15.831,0.0,0.208719,...,87.936963,0.113756,0.012940,87.928333,0.609167,0,0,1,0,0
ID_test_10,29.850000,17.108333,22.144421,2.943047,8.661528,21.458333,12.741667,1.327,0.0,0.040190,...,90.521357,0.136316,0.018582,90.536667,0.670000,0,0,0,1,0
ID_test_100,27.325000,18.633333,22.361639,2.480003,6.150415,21.616667,8.691667,3.604,0.0,0.035686,...,87.754848,0.125583,0.015771,87.726667,0.518333,0,0,1,0,0
ID_test_1000,31.091667,16.908333,23.560243,4.136978,17.114585,22.825000,14.183333,0.000,0.0,0.000000,...,90.475030,0.161193,0.025983,90.487273,0.692500,0,0,0,1,0


## Splitting the dataset into the Training set and Test set

In [16]:
from sklearn.model_selection import train_test_split
x_trainer, x_tester, y_trainer, y_tester = train_test_split(x_train, y_train, test_size=0.20)
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

reg = RandomForestRegressor(n_estimators = 150, random_state=32)
reg.fit(x_trainer, y_trainer.iloc[:, 1])
cvs = cross_val_score(reg, x_trainer, y_trainer.iloc[0:, 1], cv=5)

print(cvs)
print(np.mean(cvs))

[0.57425344 0.59231212 0.49244461 0.60403929 0.51477331]
0.5555645524231468


In [82]:
hold2 = pd.DataFrame(data = y_tester, index = train['ID'])
hold2

,ID,target
ID,,
ID_train_0,NaN,NaN
ID_train_1,NaN,NaN
ID_train_10,NaN,NaN
ID_train_100,NaN,NaN
ID_train_1000,NaN,NaN
...,...,...
ID_train_9995,NaN,NaN
ID_train_9996,NaN,NaN
ID_train_9997,NaN,NaN


In [22]:
testid = test.index
testid

Index(['ID_test_0', 'ID_test_1', 'ID_test_10', 'ID_test_100', 'ID_test_1000',
       'ID_test_1001', 'ID_test_1002', 'ID_test_1003', 'ID_test_1004',
       'ID_test_1005',
       ...
       'ID_test_990', 'ID_test_991', 'ID_test_992', 'ID_test_993',
       'ID_test_994', 'ID_test_995', 'ID_test_996', 'ID_test_997',
       'ID_test_998', 'ID_test_999'],
      dtype='object', name='ID', length=5035)

In [23]:
y_pred = reg.predict(x_test)
np.set_printoptions(precision=2)
y_pred

array([ 45.4 , 194.98,  81.46, ...,  36.85,  73.2 ,  96.68])

,ID,target
0,ID_test_0,45.398640
1,ID_test_1,194.977495
2,ID_test_10,81.457650
3,ID_test_100,161.173866
4,ID_test_1000,106.324905
...,...,...
5030,ID_test_995,92.412200
5031,ID_test_996,47.167330
5032,ID_test_997,36.848781
5033,ID_test_998,73.204704


In [26]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_pred, y_tester.iloc[0:, 1]))
print('Mean Squared Error:', metrics.mean_squared_error(y_pred, y_tester.iloc[0:, 1]))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_pred, y_tester.iloc[0:, 1])))

ValueError: Found input variables with inconsistent numbers of samples: [5035, 3108]

In [37]:
d = {'ID': testid, 'target':np.round(y_pred)}
sub = pd.DataFrame(data=d)
sub = sub[['ID', 'target']]
sub


,ID,target
0,ID_test_0,45.0
1,ID_test_1,195.0
2,ID_test_10,81.0
3,ID_test_100,161.0
4,ID_test_1000,106.0
...,...,...
5030,ID_test_995,92.0
5031,ID_test_996,47.0
5032,ID_test_997,37.0
5033,ID_test_998,73.0


In [28]:
sub.to_csv("sub_1.csv",index  = False)

In [177]:
hold3 =  pd.DataFrame(y_pred)
hold3.insert(1,'ID', y_tester.iloc[0:, 0])
hold3.head(20)

,0,ID
0,89.520595,ID_train_0
1,66.271266,NaN
2,53.125059,NaN
3,31.858127,NaN
4,47.168263,NaN
5,77.708757,ID_train_10000
6,46.299923,NaN
7,36.673215,NaN
8,32.967241,NaN
9,82.929446,NaN


## Training the Random Forest Regression model on the whole dataset

## Predicting the Test set results

## Evaluating the Model Performance